<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#La-connexion-à-la-base-de-données-Oracle" data-toc-modified-id="La-connexion-à-la-base-de-données-Oracle-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>La connexion à la base de données Oracle</a></span><ul class="toc-item"><li><span><a href="#La-connexion" data-toc-modified-id="La-connexion-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>La connexion</a></span></li></ul></li><li><span><a href="#Les-regroupements" data-toc-modified-id="Les-regroupements-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Les regroupements</a></span><ul class="toc-item"><li><span><a href="#La-copy-d'un-DataFrame" data-toc-modified-id="La-copy-d'un-DataFrame-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>La copy d'un DataFrame</a></span></li><li><span><a href="#Le-groupby" data-toc-modified-id="Le-groupby-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Le groupby</a></span></li><li><span><a href="#Assign" data-toc-modified-id="Assign-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Assign</a></span></li></ul></li><li><span><a href="#Les-équijointures" data-toc-modified-id="Les-équijointures-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Les équijointures</a></span></li><li><span><a href="#Les-jointures-externes" data-toc-modified-id="Les-jointures-externes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Les jointures externes</a></span></li></ul></div>

In [1]:
import pandas as pd 
import seaborn as sns
import warnings 
import os
import cx_Oracle

from datetime import datetime
warnings.filterwarnings(action="ignore")

%pylab inline
%load_ext sql

sys.path.append(os.path.abspath('../../.'))
from dbconnect import utilisateur,connexion,password

%sql oracle+cx_oracle://$utilisateur:$password@$connexion
        
plt.style.use('seaborn-darkgrid')
sns.set(font_scale=3)

Populating the interactive namespace from numpy and matplotlib


# La connexion à la base de données Oracle

## La connexion

In [2]:
connection = cx_Oracle.connect(utilisateur, password, connexion)

La lecture d'une table et le chargement dans un DataFrame

In [3]:
query = """
            select * 
            from categories
         """
categories = pd.read_sql_query(query, connection) #df.to_sql('nom', con=engine,if_exists='replace')
categories

,CODE_CATEGORIE,NOM_CATEGORIE,DESCRIPTION
0,1,Boissons,"Boissons, cafés, thés, bières"
1,2,Condiments,"Sauces, assaisonnements et épices"
2,3,Desserts,Desserts et friandises
3,4,Produits laitiers,Fromages
4,5,Pâtes et céréales,"Pains, biscuits, pâtes et céréales"
5,6,Viandes,Viandes préparées
6,7,Produits secs,"Fruits secs, raisins, autres"
7,8,Poissons et fruits de mer,"Poissons, fruits de mer, escargots"
8,9,Conserves,"Fruits, légumes en conserve et confitures"
9,10,Viande en conserve,Viande en conserve


# Les regroupements 

In [ ]:
query = """
            select * 
            from commandes co 
                 join details_commandes dc 
                 on co.no_commande = dc.no_commande
            --FETCH FIRST 100 ROWS ONLY
         """
df = pd.read_sql_query(query, connection) #df.to_sql('nom', con=engine,if_exists='replace')
df.shape

## La copy d'un DataFrame

In [ ]:
det_comm = df.copy()
det_comm.columns

In [ ]:
det_comm.head()

## Le groupby

<img src="https://raw.githubusercontent.com/rbizoi/Ecole-IA-Strasbourg/master/images/documents/M06-03.png" width="400">



In [ ]:
%%sql  
select annee, 
     trimestre, 
     mois, 
     sum(quantite) quantite_sum, 
     avg(quantite) quantite_avg, 
     sum(port) port_sum, 
     min(port) port_min, 
     max(port) port_max, 
     avg(port) port_avg 
from commandes co 
   join details_commandes dc
      on co.no_commande = dc.no_commande 
group by annee,trimestre,mois 
order by annee,trimestre,mois

In [ ]:
gb_dc = det_comm.groupby(['annee','trimestre','mois'])

In [ ]:
gb_dc.agg({'quantite':['sum','mean'],'port':['sum','min','max','mean']}).head()

In [ ]:
gb_dc.agg({'prix_unitaire':'std'}).head()

## Assign

In [ ]:
df_dc = det_comm.assign(
                 ca_brut = det_comm.quantite*det_comm.prix_unitaire,
                 remise  = det_comm.quantite*det_comm.prix_unitaire*det_comm.remise/100,
                 ca_net  = det_comm.quantite*det_comm.prix_unitaire - 
                          (det_comm.quantite*det_comm.prix_unitaire*det_comm.remise/100)                
                ).groupby(
                          ['annee','trimestre','mois']
                         ).agg(
                               {'quantite':['sum','mean'],
                                'port'    :['sum','min','max','mean'],
                                'ca_brut' :['sum'],
                                'remise'  :['sum'],
                                'ca_net'  :['sum'],
                                 })
df_dc.head()

In [ ]:
df_dc.columns = [col[0]+'_'+col[1] for col in df_dc.columns]
df_dc.head()

In [ ]:
df_dc.reset_index().head()

# Les équijointures

<img src="https://raw.githubusercontent.com/rbizoi/Ecole-IA-Strasbourg/master/images/documents/M06-07.png" width="400">


In [ ]:
%%sql 
select count(*) from employes 

In [ ]:
%%sql 
select count(*) from commandes 

In [ ]:
%%sql 
select count(*) from clients

In [ ]:
%%sql 
select count(*) 
     from employes em 
          inner join commandes co 
               on em.no_employe = co.no_employe 
          inner join clients cl 
               on cl.code_client = co.code_client

In [ ]:
commandes = pd.read_sql_query("select * from commandes",connection)
employes  = pd.read_sql_query("select * from employes",connection)
clients  = pd.read_sql_query("select * from clients",connection)

In [ ]:
employes.merge(commandes, on='no_employe').merge(clients, on='code_client').shape

In [ ]:
employes.merge(commandes, on='no_employe').merge(clients, on='code_client').head()

# Les jointures externes 

<img src="https://raw.githubusercontent.com/rbizoi/Ecole-IA-Strasbourg/master/images/documents/M06-07.png" width="400">


In [ ]:
%sql select count(*) from employes 

In [ ]:
%sql select count(distinct no_employe) from commandes 

In [ ]:
%%sql 
select count(*) lignes,
     count(distinct em.no_employe) as emp_employes ,
     count(distinct co.no_employe) as emp_commandes 
from employes em 
          left outer join commandes co 
               on em.no_employe = co.no_employe

<img src="https://raw.githubusercontent.com/rbizoi/Ecole-IA-Strasbourg/master/images/documents/M06-08.png" width="400">

In [ ]:
%%sql 
select em.no_employe,
       sum(port) port
from employes em 
  left outer join commandes co 
       on em.no_employe = co.no_employe 
group by em.no_employe 
order by port nulls first fetch next 25 rows only

In [ ]:
%%sql 
select 
    count(*),
    count(distinct em.no_employe),
    count(distinct co.no_employe) 
from employes em 
  left outer join commandes co 
       on em.no_employe = co.no_employe 
where co.annee = 2010

Toutefois la requête précédente ne retourne pas les employés qui n’ont pas passé de commande, car la condition ``annee = 2010`` impose également l’existence d’un enregistrement dans la table **COMMANDES**.

In [ ]:
%%sql 
select count(*),
    count(distinct em.no_employe),
    count(distinct co.no_employe) 
from employes em 
  left outer join commandes co 
       on em.no_employe = co.no_employe 
where coalesce(co.annee,2010) = 2010

In [ ]:
%%sql 
select count(*),
     count(distinct em.no_employe),
     count(distinct co.no_employe) 
     from employes em 
          left outer join commandes co 
               on em.no_employe = co.no_employe 
          left join clients cl 
               on cl.code_client = co.code_client

In [ ]:
commandes = pd.read_sql_query("select * from commandes",connection)
employes  = pd.read_sql_query("select * from employes",connection).drop(columns=['pays','region'])
clients  = pd.read_sql_query("select * from clients",connection)

In [ ]:
employes.set_index('no_employe').join(commandes.set_index('no_employe'),how='inner').shape

In [ ]:
employes.set_index('no_employe').join(commandes.set_index('no_employe'),how='left').shape

Avec join c'est un peut casse tête à mon avis 

In [ ]:
employes.set_index(
                   'no_employe'
                   ).join(
                          commandes.set_index(
                                              'no_employe'
                                              ),
                           how='left'
                          ).reset_index().set_index(
                                                    'code_client'
                                                   ).join(
                                                          clients.set_index(
                                                                            'code_client'
                                                                            ),
                                                          how='left').shape

In [ ]:
employes.merge(commandes,on='no_employe',how='left').merge(clients,on='code_client',how='left').shape